In [476]:
import pandas as pd
import numpy as np

In [477]:
df=pd.read_csv('../artifacts/glassdoor_jobs.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Manager Data Scientist,NaN,We are seeking a highly skilled and experience...,NaN,Optum,Bengaluru,NaN,NaN,NaN,NaN,NaN,NaN
1,"Lead Data Scientist, India - BCG X",₹6L – ₹9L/yr (Glassdoor est.),Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
2,Global Data Scientist Senior Manager,NaN,WHAT YOU'LL DO\n\nAs part of the Digital produ...,4.2,Boston Consulting Group,Delhi,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
3,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN
4,Principal AI or ML Engineer,NaN,"As the Principal AI/ML Engineer, you will be r...",NaN,Optum,India,NaN,NaN,NaN,NaN,NaN,NaN


In [478]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [479]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'data analyst'
    elif 'machine learning' in title.lower():
        return 'ML engineer'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'Unknown'

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'junior'
    else:
        return 'Unknown'

In [480]:
df['job_simp'] = df['Job Title'].apply(title_simplifier)
df['seniority'] = df['Job Title'].apply(seniority)

In [481]:
df['Salary Estimate'].isna().sum()

203

In [482]:
df = df.dropna(subset=['Salary Estimate'])
df.reset_index(drop=True, inplace=True)

In [483]:
df = df.dropna(subset=['Salary Estimate'])
df['Salary Estimate'] = df['Salary Estimate'].str.replace('₹', '') \
        .str.replace('L', '*100000').str.replace('T', '*1000') \
        .str.replace(r'\(Glassdoor est.\)', '', regex=True) \
        .str.replace(r'\(Employer est.\)', '', regex=True) \
        .str.replace(r'/yr', '', regex=True) \
        .str.replace(r'/mo', '*12', regex=True) \
        .str.replace(r'/hr', '*2080', regex=True)
df['avg_salary'] = df['Salary Estimate'].apply(
    lambda x: (eval(x.split('–')[0].strip()) + eval(x.split('–')[1].strip()))/2 if '–' in str(x) else eval(x))

In [484]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_simp,seniority,avg_salary
0,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),data scientist,senior,750000.0
1,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000+ Employees,1996,Company - Public,Internet & Web Services,Information Technology,$5 to $10 billion (USD),ML engineer,senior,900000.0
2,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000+ Employees,1690,Company - Public,Banking & Lending,Finance,$10+ billion (USD),data scientist,Unknown,950000.0
3,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,data scientist,Unknown,3050000.0
4,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,126000.0


In [485]:
df['Company Name'].value_counts()

Company Name
Innovecture                          68
Boston Consulting Group              67
Barclays                             34
Excellent Opportunity                34
IPCS Global solutions pvt ltd        34
S&P Global                           34
Indicraft Vintage Private Limited    34
Applied Materials                    34
Expedia Partner Solutions            34
Accusaga                             33
Nithminds Private limited            33
Exponentia.ai                        33
Arch Systems, LLC                    33
WAHY LAB SOLUTIONS                   33
objectways                           33
AIBridge ML                          33
Blackcoffer (OPC) Pvt. Ltd           33
Louis Dreyfus Company                32
Michelin                             32
Drimlite                             32
MacroCosmos Creations                32
Saaki Argus & Averil Consulting      32
Name: count, dtype: int64

In [486]:
df['Type of ownership'].isna().sum()

265

In [487]:
df['Type of ownership'].value_counts()

Type of ownership
Company - Private                 267
Company - Public                  168
Unknown                            33
Subsidiary or Business Segment     32
Private Practice / Firm            32
Name: count, dtype: int64

In [488]:
df['Industry'].isna().sum()

265

In [489]:
df['Industry'].value_counts()

Industry
Information Technology Support Services    134
Business consulting                         67
--                                          66
Internet & Web Services                     34
Banking & Lending                           34
Research and development                    34
Electronics Manufacturing                   34
Computer Hardware Development               33
Crop Production                             32
Machinery Manufacturing                     32
HR Consulting                               32
Name: count, dtype: int64

In [490]:
df['Sector'].isna().sum()

265

In [491]:
df['Sector'].value_counts()

Sector
Information Technology          201
Management and consulting       101
Manufacturing                    66
--                               66
Finance                          34
Agriculture                      32
Human resources and staffing     32
Name: count, dtype: int64

In [492]:
df[['Type of ownership', 'Industry', 'Sector','Revenue','Size']] = df[['Type of ownership', 'Industry', 'Sector','Revenue', 'Size']].fillna('Unknown')

In [493]:
df['Revenue'].value_counts()

Revenue
Unknown                     265
$10+ billion (USD)          166
Unknown / Non-Applicable    164
$5 to $10 billion (USD)     101
$5 to $25 million (USD)     101
Name: count, dtype: int64

In [494]:
df['Size'].value_counts()

Size
Unknown                 298
10000+ Employees        267
51 to 200 Employees     133
1 to 50 Employees        66
201 to 500 Employees     33
Name: count, dtype: int64

In [495]:
def clean_size(df):
    df['Size_Upper'] = df['Size'].str.replace('10000+ Employees', 'large') \
                            .str.replace('201 to 500 Employees', 'high') \
                            .str.replace('51 to 200 Employees', 'med') \
                            .str.replace('1 to 50 Employees', 'low') \
                            .str.strip()


def clean_founded(df):
    df['Founded'] = pd.to_numeric(df['Founded'], errors='coerce')
    df['Age'] = 2024 - df['Founded'].dropna().astype(int)

def clean_industry_sector_ownership(df):
    df['Industry'] = df['Industry'].apply(lambda x: 'unknown' if x == '--' else x)
    df['Sector'] = df['Sector'].apply(lambda x: 'unknown' if x == '--' else x)
    df['Ownership'] = df['Type of ownership'].str.replace('Company - Private', 'private') \
                            .str.replace('Company - Public', 'publlic') \
                            .str.replace('Subsidiary or Business Segment', 'subsidiary') \
                            .str.replace('Private Practice / Firm', 'firm') \
                            .str.strip()
 
def clean_revenue(df):
    df['Revenue_Upper'] = df['Revenue'].str.replace(r'\$5 to \$10 billion \(USD\)', 'med', regex=True) \
                                .str.replace(r'\$10\+ billion \(USD\)', 'high', regex=True) \
                                .str.replace(r'\$5 to \$25 million \(USD\)', 'low', regex=True) \
                                .str.replace(r'Unknown / Non-Applicable', 'Unknown', regex=True) \
                                .str.strip()


In [496]:
clean_size(df)
clean_founded(df)
clean_industry_sector_ownership(df)
clean_revenue(df)

In [497]:
df['Revenue_Upper'].isna().sum()

0

In [498]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_simp,seniority,avg_salary,Size_Upper,Age,Ownership,Revenue_Upper
0,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963.0,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),data scientist,senior,750000.0,large,61.0,private,med
1,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000+ Employees,1996.0,Company - Public,Internet & Web Services,Information Technology,$5 to $10 billion (USD),ML engineer,senior,900000.0,large,28.0,publlic,med
2,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000+ Employees,1690.0,Company - Public,Banking & Lending,Finance,$10+ billion (USD),data scientist,Unknown,950000.0,large,334.0,publlic,high
3,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,Unknown,NaN,Unknown,Unknown,Unknown,Unknown,data scientist,Unknown,3050000.0,Unknown,NaN,Unknown,Unknown
4,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,Unknown,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,126000.0,Unknown,NaN,Unknown,Unknown


In [499]:
df = df.drop(['Job Title', 'Salary Estimate', 'Job Description', 'Size', 'Founded', 'Revenue','Type of ownership'], axis=1)

In [500]:
df.head()

,Rating,Company Name,Location,Industry,Sector,job_simp,seniority,avg_salary,Size_Upper,Age,Ownership,Revenue_Upper
0,4.2,Boston Consulting Group,Gurgaon,Business consulting,Management and consulting,data scientist,senior,750000.0,large,61.0,private,med
1,3.8,Expedia Partner Solutions,Gurgaon,Internet & Web Services,Information Technology,ML engineer,senior,900000.0,large,28.0,publlic,med
2,3.9,Barclays,Chennai,Banking & Lending,Finance,data scientist,Unknown,950000.0,large,334.0,publlic,high
3,NaN,Excellent Opportunity,Remote,Unknown,Unknown,data scientist,Unknown,3050000.0,Unknown,NaN,Unknown,Unknown
4,NaN,IPCS Global solutions pvt ltd,India,Unknown,Unknown,Unknown,Unknown,126000.0,Unknown,NaN,Unknown,Unknown


In [501]:
df.isna().sum()

Rating           265
Company Name       0
Location           0
Industry           0
Sector             0
job_simp           0
seniority          0
avg_salary         0
Size_Upper         0
Age              364
Ownership          0
Revenue_Upper      0
dtype: int64

In [502]:
df.to_csv("../artifacts/data_clean.csv", index=False)